In [2]:
import pandas as pd
import numpy as np
import requests
import os
import json
import time

<font size='5'><b>1. Data Gathering</b></font>

In [3]:
#create dataframe by reading csv
df = pd.read_csv("twitter-archive-enhanced.csv")
df.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,8.924210e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,8.921770e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


Need to create and save a file down. Also need to creat the .txt file for the data.m

In [4]:
# we know the url
tweet_image_prediction_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

# request from the url
folder = "data"
r = requests.get(tweet_image_prediction_url)
if r.status_code == requests.codes.ok:
    # to make sure to use utf-8 encoding
    r.encoding = 'utf-u' 

    # to store the file as a TSV file
    if not os.path.exists(folder):
        os.makedirs(folder)

    with open(os.path.join(folder, 'tweet_image_predictions.txt'), 'w') as infile:
        # write the text into the txt file
        infile.write(r.text)

In [5]:
#open tsv file
images = pd.read_table('data/tweet_image_predictions.txt', sep='\t')
images.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [6]:
#creating an API object so we can gather Twitter data

import tweepy

consumer_key = 'Iorx4UF5DfzdCXoDNI3LA4zKX'
consumer_secret = 'gFELLACIIcPti3OxfHvF9SjfQYRkYgZE0Pr0zOBDQxexCHPvqe'
access_token = '2168888629-o9AyvMXXBFRPNnDIhTDjNQnAXQZI5esPBsX6IiD'
access_secret = 'iln7CAWqMxYGPZzBRlItDWlCVBosRXkVvmQqwJNhpQ0ZG'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [7]:
# create api object
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True,
                 parser=tweepy.parsers.JSONParser())

In [8]:
# get the tweet IDs from the we_rate_dogs and image predictions dataframes
tweet_IDs = np.array(df['tweet_id'].values, dtype = str)
tweet_IDs = np.concatenate((np.array(images['tweet_id'].values,
                                     dtype = str), tweet_IDs))
tweet_IDs = np.unique(tweet_IDs)

In [9]:
# get the tweets
tweets = {}

for tweet_id in tweet_IDs:
    start = time.time()
    try:
        tweets[tweet_id] = api.get_status(tweet_id, tweet_mode = 'extended')
    except tweepy.TweepError as e:
        print(e)
    end = time.time()
    request_time = end - start
    print("time required: ", request_time)

[{'code': 144, 'message': 'No status found with that ID.'}]
time required:  0.18474078178405762
[{'code': 144, 'message': 'No status found with that ID.'}]
time required:  0.17314839363098145
[{'code': 144, 'message': 'No status found with that ID.'}]
time required:  0.1775975227355957
[{'code': 144, 'message': 'No status found with that ID.'}]
time required:  0.14785265922546387
[{'code': 144, 'message': 'No status found with that ID.'}]
time required:  0.14812684059143066
[{'code': 144, 'message': 'No status found with that ID.'}]
time required:  0.16954255104064941
[{'code': 144, 'message': 'No status found with that ID.'}]
time required:  0.18435287475585938
[{'code': 144, 'message': 'No status found with that ID.'}]
time required:  0.1760542392730713
[{'code': 144, 'message': 'No status found with that ID.'}]
time required:  0.1802370548248291
[{'code': 144, 'message': 'No status found with that ID.'}]
time required:  0.1394202709197998
[{'code': 144, 'message': 'No status found w

In [10]:
# store each tweet of JSON data in a file called tweet_json.txt file
with open(os.path.join(folder, 'tweet_json.txt'), 'w', encoding='utf-8') as outfile:
    for key, value in tweets.items():
        json.dump(value, outfile)
        outfile.write('\n') # data should be written to its own line   

In [11]:
with open('tweet_json.txt', 'w') as file:
    json.dump(tweets, file)

In [12]:
with open('tweet_json.txt') as file:
    data = json.load(file)
    
df_list = []

for tweet_id in data.keys():
    retweets = data[tweet_id]['retweet_count']
    favorites = data[tweet_id]['favorite_count']# + data[tweet_id]['favourites_count']
    df_list.append({'tweet_id': tweet_id,
                        'retweets': retweets,
                        'favorites': favorites})
    
tweets_df = pd.DataFrame(df_list, columns = ['tweet_id', 'retweets', 'favorites'])
tweets_df.sample(5)

,tweet_id,retweets,favorites
1361,761334018830917632,1492,5319
1736,822489057087389700,6613,18653
1926,860184849394610176,5641,16249
1193,740359016048689152,875,3332
1285,750868782890057730,1402,4883


Assessment

3 dataframes
df = text, rating, dog category
tweet_df = has the retweets and favorites
images = has the neural network results

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null float64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(5), int64(2), 

In [14]:
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [15]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 3 columns):
tweet_id     2061 non-null object
retweets     2061 non-null int64
favorites    2061 non-null int64
dtypes: int64(2), object(1)
memory usage: 48.4+ KB


In [16]:
df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455128e+17,2.015385e+16,7.720221e+17,1.241437e+16,13.126486,10.455433
std,6.856706e+16,7.583419e+16,1.253546e+17,6.236131e+16,9.597227e+16,45.876648,6.745237
min,6.660210e+17,6.660000e+17,1.185634e+07,6.660000e+17,7.832140e+05,0.000000,0.000000
25%,6.783992e+17,6.760000e+17,3.086374e+08,7.190000e+17,4.196984e+09,10.000000,10.000000
50%,7.196275e+17,7.035000e+17,4.196984e+09,7.800000e+17,4.196984e+09,11.000000,10.000000
75%,7.993375e+17,8.260000e+17,4.196984e+09,8.200000e+17,4.196984e+09,12.000000,10.000000
max,8.924210e+17,8.860000e+17,8.410000e+17,8.870000e+17,7.870000e+17,1776.000000,170.000000


In [17]:
images.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [18]:
tweets_df.describe()

,retweets,favorites
count,2061.000000,2061.000000
mean,2641.647744,8102.197477
std,4617.256275,12240.688319
min,11.000000,0.000000
25%,563.000000,1546.000000
50%,1247.000000,3569.000000
75%,3031.000000,10112.000000
max,79305.000000,158979.000000


In [19]:
df_clean=df.copy()
images_clean=images.copy()
tweets_df_clean=tweets_df.copy()

In [20]:
tweets_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 3 columns):
tweet_id     2061 non-null object
retweets     2061 non-null int64
favorites    2061 non-null int64
dtypes: int64(2), object(1)
memory usage: 48.4+ KB


In [29]:
images_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


In [30]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(2), o

## go back through
tweets_df_clean.tweet_id = df_clean.tweet_id.astype(str)
images_clean.tweet_id = images_clean.tweet_id.astype(str)
df_clean.tweet_id = df_clean.tweet_id.astype(str)
merged_df_clean = pd.merge(df_clean, tweets_df_clean,
                        how = 'inner', on = 'tweet_id')
merged_df_clean = pd.merge(merged_df_clean, images_clean,
                        how = 'inner', on = 'tweet_id')

In [32]:
merged_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 30 columns):
tweet_id                      0 non-null object
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     0 non-null object
source                        0 non-null object
text                          0 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 0 non-null object
rating_numerator              0 non-null int64
rating_denominator            0 non-null int64
name                          0 non-null object
doggo                         0 non-null object
floofer                       0 non-null object
pupper                        0 non-null object
puppo                         0 non-null object
retweets                      0 non-null int64
favorites                     0 non-null int64


In [ ]:
merged_df_clean[merged_df_clean.retweets.isnull()]

In [ ]:
#merge columns into one
merged_df = pd.melt(merged_df_clean,
                        id_vars=['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id',
                                 'timestamp', 'source', 'text', 'retweeted_status_id',
                                 'retweeted_status_user_id', 'retweeted_status_timestamp',
                                 'expanded_urls', 'rating_numerator', 'rating_denominator',
                                 'name', 'retweets', 'favorites', 'jpg_url', 'img_num', 'p1',
                                 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf',
                                 'p3_dog'],
                        value_name='dog_stage')
merged_df = merged_df.drop('variable', axis=1)

In [ ]:
# Keep one of 3 duplicates with 'None'
dup_df1 = merged_df.drop_duplicates(subset=['tweet_id', 'dog_stage']) 
dup_df1 = merged_df.drop_duplicates()

# Keep rows with dog stage
dup_pos = merged_df.duplicated(subset='tweet_id', keep = False)
dup_df2 = merged_df[dup_pos]
dup_df2 = dup_df2[dup_df2.dog_stage != 'None']

# Combine
dup_df1 = dup_df1.drop_duplicates(subset = 'tweet_id', keep = False)
df_merged = dup_df1.append(dup_df2)

In [ ]:
df_merged.head()

In [ ]:
df_merged.info()